In [1]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf
from skimage import transform, data

F:\Anaconda3.5\Anaconda\envs\tensorFlow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# Get images
X = []
for filename in os.listdir('Train_France/'):
    X.append(img_to_array(load_img('Train_France/'+filename)))
    


In [4]:
lenth=[]
width=[]
for i in range(len(X)):
    lenth.append(len(X[i]))
    width.append(len(X[i][1]))
    if len(X[i])==256:
        X[i]=X[i]
    else:
        X[i]=np.array(X[i], dtype=float)
        X[i]=transform.resize(X[i], (256, 256))

F:\Anaconda3.5\Anaconda\envs\tensorFlow\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [5]:
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X

In [9]:
#Load weights
inception = InceptionResNetV2(weights='imagenet', include_top=True)
inception.graph = tf.get_default_graph()

In [30]:
inception.save_weights("SaveModel/inception.h5")

In [11]:
embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [25]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

#Generate training data
batch_size = 10

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)


#Train model      
model.compile(optimizer='adam', loss='mse')
model.fit_generator(image_a_b_gen(batch_size), epochs=200, steps_per_epoch=13)

Epoch 1/200
13/13 [==============================] - 145s 11s/step - loss: 0.0051
Epoch 2/200
13/13 [==============================] - 125s 10s/step - loss: 0.0052
Epoch 3/200
13/13 [==============================] - 139s 11s/step - loss: 0.0049
Epoch 4/200
13/13 [==============================] - 125s 10s/step - loss: 0.0043
Epoch 5/200
13/13 [==============================] - 138s 11s/step - loss: 0.0049
Epoch 6/200
13/13 [==============================] - 125s 10s/step - loss: 0.0045
Epoch 7/200
13/13 [==============================] - 137s 11s/step - loss: 0.0049
Epoch 8/200
13/13 [==============================] - 125s 10s/step - loss: 0.0049
Epoch 9/200
13/13 [==============================] - 137s 11s/step - loss: 0.0050
Epoch 10/200
13/13 [==============================] - 125s 10s/step - loss: 0.0043
Epoch 11/200
13/13 [==============================] - 138s 11s/step - loss: 0.0048
Epoch 12/200
13/13 [==============================] - 125s 10s/step - loss: 0.0047
Epoch 13/200


13/13 [==============================] - 138s 11s/step - loss: 0.0024
Epoch 100/200
13/13 [==============================] - 125s 10s/step - loss: 0.0023
Epoch 101/200
13/13 [==============================] - 138s 11s/step - loss: 0.0023
Epoch 102/200
13/13 [==============================] - 125s 10s/step - loss: 0.0022
Epoch 103/200
13/13 [==============================] - 138s 11s/step - loss: 0.0023
Epoch 104/200
13/13 [==============================] - 125s 10s/step - loss: 0.0023
Epoch 105/200
13/13 [==============================] - 138s 11s/step - loss: 0.0024
Epoch 106/200
13/13 [==============================] - 126s 10s/step - loss: 0.0022
Epoch 107/200
13/13 [==============================] - 138s 11s/step - loss: 0.0022
Epoch 108/200
13/13 [==============================] - 125s 10s/step - loss: 0.0020
Epoch 109/200
13/13 [==============================] - 138s 11s/step - loss: 0.0021
Epoch 110/200
13/13 [==============================] - 125s 10s/step - loss: 0.0019
Epoch 

13/13 [==============================] - 126s 10s/step - loss: 0.0050
Epoch 197/200
13/13 [==============================] - 138s 11s/step - loss: 0.0048
Epoch 198/200
13/13 [==============================] - 125s 10s/step - loss: 0.0056
Epoch 199/200
13/13 [==============================] - 139s 11s/step - loss: 0.0047
Epoch 200/200
13/13 [==============================] - 127s 10s/step - loss: 0.0048


In [26]:
model.save_weights("SaveModel/cifarCnnModel.h5")

In [29]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("success")
except:
    print("fail")

success


In [35]:
color_me = []
for filename in os.listdir('Train_France/'):
    color_me.append(img_to_array(load_img('Train_France/'+filename)))


lenth=[]
width=[]
for i in range(len(color_me)):
    lenth.append(len(color_me[i]))
    width.append(len(color_me[i][1]))
    if len(color_me[i])==256:
        color_me[i]=color_me[i]
    else:
        color_me[i]=np.array(color_me[i], dtype=float)
        color_me[i]=transform.resize(color_me[i], (256, 256))    

color_me = np.array(color_me, dtype=float)
gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
color_me_embed = create_inception_embedding(gray_me)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))


# Test model
output = model.predict([color_me, color_me_embed])
output = output * 128


F:\Anaconda3.5\Anaconda\envs\tensorFlow\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [36]:
# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    cur=transform.resize(cur, (lenth[i], width[i]))
    imsave("result_France/img_"+str(i)+".png", lab2rgb(cur))

F:\Anaconda3.5\Anaconda\envs\tensorFlow\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
F:\Anaconda3.5\Anaconda\envs\tensorFlow\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
